In [0]:
from dUI import dUI, CONTEXT, CONTEXT_RAW
from dUI.utils import *
from pprint import pprint

tw = dbutils.widgets.text
dw = dbutils.widgets.dropdown
dwg = dbutils.widgets.get

tw('database_name', 'default', 'Database Name')
tw('sql_folder', f'/Workspace/Repos/{CONTEXT.user}/dUI/queries/tpcds_2.13', 'SQL Folder')
tw('host', CONTEXT.host, 'API Hostname')
tw('tscope', '', 'Token Scope')
tw('tkey', '', 'Token key')

tw('cluster_id', '', 'SQL Endpoint ID')

csizes = ['2X-Small', 'X-Small', 'Small', 'Medium', 'Large', 'X-Large', '2X-Large', '3X-Large', '4X-Large']
dw('Size', 'Medium', csizes)
tw('clusters_min', '1', 'Min Clusters')
tw('clusters_max', '1', 'Max Clusters')
dw('Serverless', 'False', ['True', 'False'])
dw('Channel', 'Current', ['Current', 'Preview'])

# dbutils.widgets.text('parallel_conn','5','Parallel Connections')
# dbutils.widgets.text('re_peats','10','Query Repeats')

In [0]:
database = dwg('database_name')
sql_dir = dwg('sql_folder')
host = dwg('host')
# token = CONTEXT_RAW['extraContext']['api_token']

try:
    token = dbutils.secrets.get(scope=dwg('tscope'), key=dwg('tkey'))
    print(f"Token retrieval success")
except BaseException as e:
    print("Something went wrong retrieving PA token.")
    print("PAT required in order to use sqlstorm.")
    print(e)

if (cluster_id := dwg('cluster_id')) == '':
    print('Creating SQL Endpoint')
    ep_cfg = EndpointCfg(
        size=dwg('Size'),
        clusters_min=dwg('clusters_min'),
        clusters_max=dwg('clusters_max'),
        serverless=True if dwg('Serverless') == 'True' else False,
        channel="CHANNEL_NAME_CURRENT" if dwg('Channel') == 'Current' else "CHANNEL_NAME_PREVIEW"
    )
    cluster_id = create_ep(token, host, ep_cfg)

ep_details = get_or_del_ep(token, host, cluster_id)
pprint(ep_details)
  

# parallel_conn = dbutils.widgets.get('parallel_conn')
# re_peats = dbutils.widgets.get('re_peats')

Token retrieval success
Creating SQL Endpoint
new cluster_id: 4d1313e2cc28cad2
ntuple(id='4d1313e2cc28cad2', name='Storm Cloud', size='MEDIUM', cluster_size='Medium', min_num_clusters=1, max_num_clusters=1, auto_stop_mins=15, auto_resume=True, creator_name='pavan.dendi@databricks.com', creator_id=8008172456975791, tags=tags(), spot_instance_policy='COST_OPTIMIZED', enable_photon=True, channel=channel(name='CHANNEL_NAME_CURRENT'), enable_serverless_compute=False, num_clusters=0, num_active_sessions=0, state='STARTING', jdbc_url='jdbc:spark://adb-2290777133481849.9.azuredatabricks.net:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/endpoints/4d1313e2cc28cad2;', odbc_params=odbc_params(hostname='adb-2290777133481849.9.azuredatabricks.net', path='/sql/1.0/endpoints/4d1313e2cc28cad2', protocol='https', port=443), health=health(status='HEALTHY'))


In [0]:
# from dUI import dUI
jdbc = simba_jdbc(token, ep_details.odbc_params)
query_test = Query("test", "SELECT 1")
# queries = utils.read_sql_folder(sql_dir, info=True)
dUI.run(jdbc, [query_test])

Writing yaml config file to: /dbfs/tmp/dbstress/config.yaml
---
unit_name: test
query: "/* test  @@gen_query_id@@ */
SELECT 1"
uri: "jdbc:spark://adb-2290777133481849.9.azuredatabricks.net:443/{database};transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/endpoints/4d1313e2cc28cad2;UID=token;PWD=[REDACTED];UseNativeQuery=1"
driver_class: com.simba.spark.jdbc.Driver
username: username
password: password
parallel_connections: 1
repeats: 1
connection_timeout: 3000



In [0]:
# host = "adb-2290777133481849.9.azuredatabricks.net"
# cluster_id = "fe5412cd68e9d437"
# token = context.extraContext.api_token
# apiurl = f"https://{host}/api/2.0/sql/endpoints/"
# headers = {"Authorization": f"Bearer {token}"}
# response = requests.delete(apiurl+cluster_id, headers=headers).json()
print(cluster_id)
response = get_or_del_ep(token, host, cluster_id, delete=True)
pprint(response)

4d1313e2cc28cad2
{}


In [0]:
dbutils.widgets.removeAll()